In [1]:
import pandas as pd


class Loader:
    def __init__(self, path: str = "", pickled_path: str = ""):
        self.path = path
        self.pickled_path = ""
        self.df: pd.DataFrame = None

    def load(self):
        self.df = pd.read_csv(self.path)

    def activate_preprocessing(self, pre_processing_functions):
        for fun in pre_processing_functions:
            self.df = fun(self.df)

    def pickle_data(self):
        if self.df is not None:
            self.df.to_pickle(self.path + ".pickled")
            self.pickled_path = self.path + ".pickled"

    def load_pickled(self):
        self.df = pd.read_pickle(self.pickled_path)

    def get_data(self) -> pd.DataFrame:
        if self.df is not None:
            return self.df


In [ ]:
train_feats_loader = Loader(path="C:/Users/Tomer/Desktop/IML.Hackathon/Mission2_Breast_Cancer/train.feats.csv")
train_feats_loader.load()
train_feats = train_feats_loader.get_data()

In [3]:
train_labels_0_loader = Loader(path="C:/Users/Tomer/Desktop/IML.Hackathon/Mission2_Breast_Cancer/train.labels.0.csv")
train_labels_0_loader.load()
train_labels_0 = train_labels_0_loader.get_data()

In [4]:
train_labels_1_loader = Loader(path="C:/Users/Tomer/Desktop/IML.Hackathon/Mission2_Breast_Cancer/train.labels.1.csv")
train_labels_1_loader.load()
train_labels_1 = train_labels_0_loader.get_data()

In [78]:
df = train_feats.iloc[:, list(range(1,11))]

In [ ]:
## אבחנה-Ivi -Lymphovascular invasion
df["אבחנה-Ivi -Lymphovascular invasion"].replace({
                                                "nan": "0","none": "0","+": "1","extensive": "1","-": "0",
                                                "No":"0", "(-)":"0","NO": "0","(+)": "1","neg": "0",
                                                "not": "0","pos": "0","yes":"1","no":"0", None:"0"
                                                }, inplace=True)

In [ ]:
## אבחנה-KI67 protein
non_null = df.loc[~(df["אבחנה-KI67 protein"].isnull())]
percentage_num = non_null.loc[non_null["אבחנה-KI67 protein"].str.match("^\d+(?:\.\d+)?%$")]
percentage_num["אבחנה-KI67 protein"] = (percentage_num["אבחנה-KI67 protein"].str.replace("%","").astype(float))/100

df.update(percentage_num)

percentage_range = non_null.loc[non_null["אבחנה-KI67 protein"].str.match("^\d+(?:\.\d+)?%-\d+(?:\.\d+)?%$")]
percentage_range["אבחנה-KI67 protein"] =(percentage_range["אבחנה-KI67 protein"].str.extract("^\d+(?:\.\d+)?%-\d+(?:\.\d+)?%$")
# to calculate average between the two numbers

df.update(percentage_range)

# many unique values, to drop very unique, deal with null values and convert others

In [ ]:
## 'אבחנה-Her2'
df['אבחנה-Her2']
# many unique values, to drop very unique, deal with null values and convert others

In [ ]:
# to convert to one-hot encoding
df[' Hospital']

# to convert to ordinal encoding (certain)
df['אבחנה-Basic stage']
df['אבחנה-Histopatological degree'] # deal with null values


# to convert to ordinal encoding (uncertain)
df['User Name']
df['אבחנה-Histological diagnosis']

# not sure how to deal
df['אבחנה-Diagnosis date']